In [1]:
import spacy 
from spacy.pipeline import EntityRuler
from spacy import displacy
import pandas as pd
import numpy as np 


In [23]:
df = pd.read_csv("../data/sample_resumes/Resume/resumeDataSet2_transformed_filtered.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()


,ID,Resume_str,Resume_html,Category
337,878,Hobbies â¢ Playing Chess â¢ Solving Rubik's ...,NaN,Blockchain
104,452,Education Details \r\nJuly 2008 to February 20...,NaN,SAP Developer
139,583,Education Details \r\nJune 2013 to June 2016 D...,NaN,Python Developer
39,359,TECHNICAL SKILLS Programming Languages: Java (...,NaN,Java Developer
314,855,"KEY SKILLS: Programing languages: C, C++, Pyth...",NaN,Blockchain


In [24]:
# import en_core_web_lg


# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm", disable=['ner'])
# nlp.add_pipe("ner")

ruler = nlp.add_pipe("entity_ruler", before='ner')

# ner = nlp.add_pipe("ner")

#Skills for entities 

SweSkills = ["Python", "Java", "JavaScript", "C++", "React", "Angular", "Node.js", "Git", "CI/CD"]
DsSkills = ["Python", "R", "SQL", "Machine Learning", "Data Analysis", "Statistics", "TensorFlow", "PyTorch"]
DoSkills = ["Linux", "Docker", "Kubernetes", "AWS", "Azure", "CI/CD", "Jenkins", "Terraform", "Ansible"]

patterns = []   

for skill in SweSkills:
    patterns.append({"label": "SWE", "pattern": skill})
for skill in DsSkills:
    patterns.append({"label": "DATASCIENCE", "pattern": skill})
for skill in DoSkills:
    patterns.append({"label": "DEVOPS", "pattern": skill})

ruler.add_patterns(patterns)

doc = nlp("Hello I love me some Angular some python, some other shit too like Terraform, DEVOPS, and idk some other shit and data sciennce")

# resume_text = data['']

for ent in doc.ents:
    print(ent.text, ent.label_)

# print(data['Resume_str'].iloc[0])

Angular SWE
Terraform DEVOPS


In [25]:
from spacy.training import Example

# Testing this out rq
train_data = []
for text in data['Resume_str']:
    doc = nlp(text)
    entities = []

    for ent in doc.ents:
        entities.append([ent.start_char, ent.end_char, ent.label_])
    train_data.append([text, {"entities": entities }])

print(train_data)

[["Hobbies â\x80¢ Playing Chess â\x80¢ Solving Rubik's Cube â\x80¢ Watching Series Languages â\x80¢ English â\x80¢ Hindi â\x80¢ Marathi Education Details \r\nJanuary 2014 to January 2017 Bachelorâ\x80\x99s Degree Information Technology, First Class Pune, Maharashtra JSPMâ\x80\x99s Jayawantrao Sawant College of Engineering\r\nJanuary 2010 to January 2014 Diploma Information Technology, First Class Nashik, Maharashtra K. K. Wagh Polytechnic\r\nJanuary 2010 SSC, First Class  Nashik, Maharashtra New Era English School\r\nBlockchain Developer \r\n\r\nBlockchain Developer - Corpcloud Global Services Pvt. Ltd\r\nSkill Details \r\nBlockchain- Exprience - 6 months\r\nSmart Contracts- Exprience - 6 months\r\nDApps- Exprience - 6 months\r\nMEAN Stack- Exprience - 12 monthsCompany Details \r\ncompany - Corpcloud Global Services Pvt. Ltd.\r\ndescription - â\x80¢ Worked productively with the Team to identify requirements and proposed ideas on enhancing the product.\r\nâ\x80¢ Developing and managing 

In [26]:
from pprint import pprint

for i, (text, annot) in enumerate(train_data):  # limit to first 5 for clarity
    print(f"\nExample {i+1}:")
    print("Text preview:", text[:200], "...\n")  # first 200 chars

    for start, end, label in annot["entities"]:
        span_text = text[start:end]
        print(f"  - '{span_text}' ({label}) [{start}:{end}]")


# pprint(train_data) 


Example 1:
Text preview: Hobbies â¢ Playing Chess â¢ Solving Rubik's Cube â¢ Watching Series Languages â¢ English â¢ Hindi â¢ Marathi Education Details 
January 2014 to January 2017 Bachelorâs Degree Information Tech ...


Example 2:
Text preview: Education Details 
July 2008 to February 2012 BE COMPUTER SCIENCE Nagpur, Maharashtra NAGPUR UNIVERSITY
SAP HANA Developer 

SAP HANA Developer - Schlumberger
Skill Details 
SAP HANA- Exprience  ...

  - 'SQL' (DATASCIENCE) [3463:3466]

Example 3:
Text preview: Education Details 
June 2013 to June 2016 Diploma Computer science Pune, Maharashtra Aissms
June 2016 BE pursuing Computer science Pune, Maharashtra Anantrao pawar college of Engineering & Research  ...

  - 'Python' (SWE) [208:214]

Example 4:
Text preview: TECHNICAL SKILLS Programming Languages: Java (Servlet, JSP, Spring Boot). Web Technology: HTML5, CSS3, Bootstrap, JavaScript, JQuery, Ajax, AngularJs. Database: MySQL. IDE and Tool: Eclipse, spring to ...

  - 'Java' (SWE)